# MedDSL-Lite Trace Demo

This notebook demonstrates the MedDSL-Lite system by running a diabetic retinopathy triage case through the rule engine and generating explanations.


In [ ]:
import sys
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

# Add the parent directory to the path to import mddsl
sys.path.append(str(Path.cwd().parent))

from mddsl import execute, explain, SnippetRetriever
from mddsl.interpreter import load_dsl_from_file


## Load Case Data and Rules


In [ ]:
# Load the DSL rules
dsl = load_dsl_from_file("../rules/dr_triage_v1.yaml")
print("Loaded DSL with", len(dsl["nodes"]), "nodes")
print("Profile:", dsl["meta"]["profile"])
print("Version:", dsl["meta"]["version"])


In [ ]:
# Load sample cases
cases = []
with open("../data/cases_small.jsonl", "r") as f:
    for line in f:
        if line.strip():
            cases.append(json.loads(line))

print(f"Loaded {len(cases)} cases")

# Display first case
print("\nFirst case:")
print(json.dumps(cases[0], indent=2))


## Execute DSL on Sample Case


In [ ]:
# Select a case for demonstration (severe NPDR with vision reduced)
demo_case = cases[0]  # Severe NPDR case

print("Demo case:")
print(json.dumps(demo_case, indent=2))

# Execute the DSL
actions, trace = execute(dsl, demo_case)

print("\nActions taken:")
for i, action in enumerate(actions, 1):
    print(f"{i}. {action}")


## Generate Explanation


In [ ]:
# Initialize snippet retriever
retriever = SnippetRetriever("../snippets")

# Generate explanation
explanation = explain(demo_case, actions, trace, retriever)

print("=== EXPLANATION ===\n")
print("Case Summary:")
print(json.dumps(explanation["case_summary"], indent=2))

print("\nRule Trace:")
for bullet in explanation["rule_trace"]:
    print(bullet)

print("\nActions:")
print(json.dumps(explanation["actions_json"], indent=2))

if explanation["citations"]:
    print("\nCitations:")
    for citation in explanation["citations"]:
        print(f"  • {citation}")

print("\nProse:")
print(explanation["prose"])
